In [4]:
import os

import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import librosa
import librosa.display


import utils

plt.rcParams['figure.figsize'] = (17, 5)

In [14]:
# Directory where mp3 are stored.
AUDIO_DIR = '/Users/annelahann/neue-fische/RiffRefine/data/fma_small'
#AUDIO_DIR = os.environ.get('AUDIO_DIR')

# Load metadata and features.
tracks = utils.load('data/tracks.csv')
genres = utils.load('data/genres.csv')
features = utils.load('data/features.csv')
echonest = utils.load('data/echonest.csv')

np.testing.assert_array_equal(features.index, tracks.index)
assert echonest.index.isin(tracks.index).all()

tracks.shape, genres.shape, features.shape, echonest.shape


<class 'pandas.core.frame.DataFrame'>
Index: 106574 entries, 2 to 155320
Data columns (total 52 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   (album, comments)            106574 non-null  int64         
 1   (album, date_created)        103045 non-null  datetime64[ns]
 2   (album, date_released)       70294 non-null   datetime64[ns]
 3   (album, engineer)            15295 non-null   object        
 4   (album, favorites)           106574 non-null  int64         
 5   (album, id)                  106574 non-null  int64         
 6   (album, information)         83149 non-null   category      
 7   (album, listens)             106574 non-null  int64         
 8   (album, producer)            18060 non-null   object        
 9   (album, tags)                106574 non-null  object        
 10  (album, title)               105549 non-null  object        
 11  (album, tracks)              10

In [18]:
#flatten column hierarchy of tracks table
tracks.columns = tracks.columns.map('_'.join)
tracks = tracks.reset_index()


In [19]:
#filter tracks.csv for subset==small 
tracks_small = tracks[tracks['set_subset'] == 'small']

#save tracks_small as separate csv
tracks_small.to_csv('data/tracks_small.csv')

In [20]:
#flatten column hierarchy of features table
features.columns = features.columns.map('_'.join)
features = features.reset_index()

In [21]:
#merge tracks_small with features table via left join on tracks_small
df_merged = tracks_small.merge(features, on='track_id', how='left')

In [28]:
df_merged.shape
df_merged.describe().T

,count,mean,min,25%,50%,75%,max,std
track_id,8000.0,84126.770125,2.0,49063.5,89281.0,122545.0,155066.0,43297.426287
album_comments,8000.0,0.538625,0.0,0.0,0.0,0.0,12.0,1.392211
album_date_created,8000,2013-05-09 17:58:33.380749824,2008-11-26 01:44:45,2011-04-23 11:51:48,2013-08-03 00:19:48,2015-07-04 07:46:50,2017-03-24 19:40:34,NaN
album_date_released,5337,2011-11-03 12:39:15.480607232,1930-01-01 00:00:00,2009-11-03 00:00:00,2012-08-27 00:00:00,2014-10-31 00:00:00,2021-03-01 00:00:00,NaN
album_favorites,8000.0,2.02525,0.0,0.0,1.0,2.0,40.0,3.825728
...,...,...,...,...,...,...,...,...
zcr_mean_01,8000.0,0.050856,0.00349,0.032412,0.046102,0.06277,0.629665,0.028484
zcr_median_01,8000.0,0.041552,0.0,0.024414,0.036621,0.052246,0.749512,0.02708
zcr_min_01,8000.0,0.002503,0.0,0.0,0.000977,0.003418,0.170898,0.004689
zcr_skew_01,8000.0,3.519118,-3.143864,1.647383,2.721854,4.601721,36.000904,2.962467


In [30]:
df_merged.to_csv('data/tr_fea_small.csv')

In [22]:
#prepare echonest table
echonest.columns = echonest.columns.droplevel()
echonest.columns = echonest.columns.map('_'.join)
echonest = echonest.reset_index()

In [29]:
#join with df_merged
df_merged_echo = df_merged.merge(echonest, on='track_id', how='left')

df_merged_echo.shape

(8000, 820)